In [4]:
print("Please work4!!!")

from tensorflow.python.client import device_lib

device_lib.list_local_devices()


Please work4!!!


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7775933944503524094, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 203348377
 locality {
   bus_id: 1
 }
 incarnation: 11743895751215014819
 physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [5]:
from tensorflow.python.client import device_lib



def get_available_gpus():

    local_device_protos = device_lib.list_local_devices()

    return [x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU']



get_available_gpus()

['device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0']

In [5]:
import os.path
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import tensorflow as tf
with tf.device('GeForce GTX 960M'):
    


    data_dir = './data'
    
    tests.test_for_kitti_dataset(data_dir)
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    
    
    def load_vgg(sess, vgg_path):
        """
        Load Pretrained VGG Model into TensorFlow.
        :param sess: TensorFlow Session
        :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
        :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
        """
        # TODO: Implement function
        #   Use tf.saved_model.loader.load to load the model and weights    
        vgg_tag = 'vgg16'
        vgg_input_tensor_name = 'image_input:0'
        vgg_keep_prob_tensor_name = 'keep_prob:0'
        vgg_layer3_out_tensor_name = 'layer3_out:0'
        vgg_layer4_out_tensor_name = 'layer4_out:0'
        vgg_layer7_out_tensor_name = 'layer7_out:0'
        
        tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
        graph = tf.get_default_graph()
        
        image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
        
        keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
        
        layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
        
        layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
        
        layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
        
        return image_input, keep_prob, layer3_out, layer4_out, layer7_out
    
    
    tests.test_load_vgg(load_vgg, tf)
    
    print("loaded vgg model")

ValueError: Unknown attribute: 'GeForce GTX 960M' in 'GeForce GTX 960M'

In [1]:
import tensorflow as tf
import numpy as np

k=8
s=4

def same_upsample(x):
    return tf.layers.conv2d_transpose(x, 3, k, s, padding='SAME')

def valid_upsample(x):
    return tf.layers.conv2d_transpose(x, 3, k, s, padding='VALID')

x = tf.constant(np.random.randn(1, 30, 30, 3), dtype=tf.float32)
same_conv = same_upsample(x)
valid_conv = valid_upsample(x)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    resultsame = sess.run(same_conv)
    resultvalid = sess.run(valid_conv)
    
    print('Input Shape: {}'.format(x.get_shape()))
    print('Same Shape: {}'.format(resultsame.shape))
    print('Valid Shape: {}'.format(resultvalid.shape))

Input Shape: (1, 30, 30, 3)
Same Shape: (1, 120, 120, 3)
Valid Shape: (1, 124, 124, 3)


### Layers function

In [2]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import tensorflow as tf
with tf.device('/GPU:1'):
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))



    # Check TensorFlow Version
    assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
    print('TensorFlow Version: {}'.format(tf.__version__))
    
    # Check for a GPU
    if not tf.test.gpu_device_name():
        warnings.warn('No GPU found. Please use a GPU to train your neural network.')
    else:
        print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
        
    print("All done")
    
    
    def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
        """
        Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
        :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
        :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
        :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
        :param num_classes: Number of classes to classify
        :return: The Tensor for the last layer of output
        """
        # TODO: Implement function
        vgg_layer3_out = tf.layers.batch_normalization(vgg_layer3_out, name="new_vgg_layer3_out")
        vgg_layer4_out = tf.layers.batch_normalization(vgg_layer4_out, name="new_vgg_layer4_out")
        vgg_layer7_out = tf.layers.batch_normalization(vgg_layer7_out, name="new_vgg_layer7_out")
        
        print(vgg_layer3_out.get_shape().as_list())
        print(vgg_layer4_out.get_shape().as_list())
        print(vgg_layer7_out.get_shape().as_list())
    
        
        #1x1 Convolution on layer 7
        layer7_1x1 = tf.layers.conv2d(vgg_layer7_out, filters=num_classes, kernel_size=(1, 1), strides=(1, 1),
                                              name='new_layer7_1x1_out', activation=tf.nn.relu,
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                              kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
        #upsampling on layer7_1x1
        layer4_upsampled =  tf.layers.conv2d_transpose(layer7_1x1, vgg_layer4_out.get_shape().as_list()[-1], 4, 
                                                 strides= (2, 2), 
                                                 padding= 'same', 
                                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                                 kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                                 name='layer4_upsampled')
        
        #batch normalization on upsampled layer
        layer4_upsampled_bn = tf.layers.batch_normalization(layer4_upsampled, name="layer4_upsampled_bn")
        
        #1x1 Convolution on vgg_layer4_out   
        vgg_layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, filters=vgg_layer4_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                              name="vgg_layer4_1x1", activation=tf.nn.relu,
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                              kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
        #batch normalization on vgg layer
        vgg_layer4_1x1_bn = tf.layers.batch_normalization(vgg_layer4_1x1, name="vgg_layer4_1x1_bn")
        
        #Final layer 4
        layer4_out = tf.add(vgg_layer4_1x1_bn, layer4_upsampled_bn, name='layer4_out')
        
        
        # Upsampled layer 3 using final layer 4
        layer3_upsampled4 = tf.layers.conv2d_transpose(layer4_out, vgg_layer3_out.get_shape().as_list()[-1], 4, 
                                                 strides= (2, 2), 
                                                 padding= 'same', 
                                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                                 kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                                 name='layer3_upsampled')
        
        #batch normalization on upsampled layer
        layer3_upsampled4_bn = tf.layers.batch_normalization(layer3_upsampled4, name="layer3_upsampled4_bn")
        
            
        #layer3 taken from a 4x upsampling from layer7_1x1
        layer3_upsampled7 = tf.layers.conv2d_transpose(layer7_1x1, vgg_layer3_out.get_shape().as_list()[-1], 8, 
                                                 strides= (4, 4), 
                                                 padding= 'same', 
                                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                                 kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                                 name='layer3_upsampled7')
        
        layer3_upsampled7_bn = tf.layers.batch_normalization(layer3_upsampled7, name="layer3_upsampled7_bn")
        
        
        #1x1 Convolution on vgg_layer3_out
        vgg_layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, filters=vgg_layer3_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                              name="vgg_layer3_1x1", activation=tf.nn.relu,
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                              kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
        #batch normalization on vgg layer
        vgg_layer3_1x1_bn = tf.layers.batch_normalization(vgg_layer3_1x1, name="vgg_layer3_1x1_bn")
        
    
        
        #Final Layer 3
        layer3_out = tf.add_n([vgg_layer3_1x1_bn, layer3_upsampled4_bn, layer3_upsampled7_bn], name='layer3_out')
        
        
        #Last Layer
        final_layer = tf.layers.conv2d_transpose(layer3_out, num_classes, 16,  
                                                   strides= (8, 8), 
                                                   padding= 'same', 
                                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
            
        return final_layer
    
    tests.test_layers(layers)
    
    print("Done")


Default GPU Device: /device:GPU:0
TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
All done
[None, None, None, 256]
[None, None, None, 512]
[None, None, None, 4096]
Tests Passed
Done


### Train_nn function

In [4]:
print("FNJNDJND")

import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

with tf.device('/gpu:1'):
    
    # Check TensorFlow Version
    assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
    print('TensorFlow Version: {}'.format(tf.__version__))
    
    # Check for a GPU
    if not tf.test.gpu_device_name():
        warnings.warn('No GPU found. Please use a GPU to train your neural network.')
    else:
        print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
        
    print("All done")
    
    def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate):
        """
        Train neural network and print out the loss during training.
        :param sess: TF Session
        :param epochs: Number of epochs
        :param batch_size: Batch size
        :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
        :param train_op: TF Operation to train the neural network
        :param cross_entropy_loss: TF Tensor for the amount of loss
        :param input_image: TF Placeholder for input images
        :param correct_label: TF Placeholder for label images
        :param keep_prob: TF Placeholder for dropout keep probability
        :param learning_rate: TF Placeholder for learning rate
        """
        # TODO: Implement function
        ## sess.run(tf.global_variables_initializer())
    
        
        print("Training...")
        for i in range(epochs):
            print("EPOCH {} ...".format(i+1))
            for image, label in get_batches_fn(batch_size):
                _, loss = sess.run([train_op, cross_entropy_loss], 
                                   feed_dict={input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0005})
            print("Loss: = {:.3f}".format(loss))
        
        print("Training Finished!!")
            
    tests.test_train_nn(train_nn)
    
    print("Training DOne")

FNJNDJND
TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
All done


In [5]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
print("All done")

def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)
print("load_vgg")



def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    vgg_layer3_out = tf.layers.batch_normalization(vgg_layer3_out, name="new_vgg_layer3_out")
    vgg_layer4_out = tf.layers.batch_normalization(vgg_layer4_out, name="new_vgg_layer4_out")
    vgg_layer7_out = tf.layers.batch_normalization(vgg_layer7_out, name="new_vgg_layer7_out")
    
    #1x1 Convolution on layer 7
    layer7_1x1 = tf.layers.conv2d(vgg_layer7_out, filters=num_classes, kernel_size=(1, 1), strides=(1, 1),
                                          name='new_layer7_1x1_out', activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #upsampling on layer7_1x1
    layer4_upsampled =  tf.layers.conv2d_transpose(layer7_1x1, vgg_layer4_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer4_upsampled')
    
    #batch normalization on upsampled layer
    layer4_upsampled_bn = tf.layers.batch_normalization(layer4_upsampled, name="layer4_upsampled_bn")
    
    #1x1 Convolution on vgg_layer4_out   
    vgg_layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, filters=vgg_layer4_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer4_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer4_1x1_bn = tf.layers.batch_normalization(vgg_layer4_1x1, name="vgg_layer4_1x1_bn")
    
    #Final layer 4
    layer4_out = tf.add(vgg_layer4_1x1_bn, layer4_upsampled_bn, name='layer4_out')
    
    
    # Upsampled layer 3 using final layer 4
    layer3_upsampled4 = tf.layers.conv2d_transpose(layer4_out, vgg_layer3_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled')
    
    #batch normalization on upsampled layer
    layer3_upsampled4_bn = tf.layers.batch_normalization(layer3_upsampled4, name="layer3_upsampled4_bn")
    
        
    #layer3 taken from a 4x upsampling from layer7_1x1
    layer3_upsampled7 = tf.layers.conv2d_transpose(layer7_1x1, vgg_layer3_out.get_shape().as_list()[-1], 8, 
                                             strides= (4, 4), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled7')
    
    layer3_upsampled7_bn = tf.layers.batch_normalization(layer3_upsampled7, name="layer3_upsampled7_bn")
    
    
    #1x1 Convolution on vgg_layer3_out
    vgg_layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, filters=vgg_layer3_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer3_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer3_1x1_bn = tf.layers.batch_normalization(vgg_layer3_1x1, name="vgg_layer3_1x1_bn")
    

    
    #Final Layer 3
    layer3_out = tf.add_n([vgg_layer3_1x1_bn, layer3_upsampled4_bn, layer3_upsampled7_bn], name='layer3_out')
    
    
    #Last Layer
    final_layer = tf.layers.conv2d_transpose(layer3_out, num_classes, 16,  
                                               strides= (8, 8), 
                                               padding= 'same', 
                                               kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
    return final_layer

tests.test_layers(layers)

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=nn_last_layer, labels=correct_label),
                                        name="cross_entropy")
    regularization_loss = tf.losses.get_regularization_loss()
    total_loss = tf.add(cross_entropy_loss, regularization_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(total_loss)
    
    
    return logits , train_op, total_loss
tests.test_optimize(optimize)

print("one more test left")

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    ## sess.run(tf.global_variables_initializer())

    
    print("Training...")
    for i in range(epochs):
        print("EPOCH {} ...".format(i+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0005})
        print("Loss: = {:.3f}".format(loss))

tests.test_train_nn(train_nn)

print("Done")

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
All done
Tests Passed
load_vgg
Tests Passed
Tests Passed
one more test left


In [2]:
import os.path
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tqdm import tqdm
import tensorflow as tf

    # Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    

def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    vgg_layer3_out = tf.layers.batch_normalization(vgg_layer3_out, name="new_vgg_layer3_out")
    vgg_layer4_out = tf.layers.batch_normalization(vgg_layer4_out, name="new_vgg_layer4_out")
    vgg_layer7_out = tf.layers.batch_normalization(vgg_layer7_out, name="new_vgg_layer7_out")
    
    #1x1 Convolution on layer 7
    layer7_1x1 = tf.layers.conv2d(vgg_layer7_out, filters=num_classes, kernel_size=(1, 1), strides=(1, 1),
                                          name='new_layer7_1x1_out', activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #upsampling on layer7_1x1
    layer4_upsampled =  tf.layers.conv2d_transpose(layer7_1x1, vgg_layer4_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer4_upsampled')
    
    #batch normalization on upsampled layer
    layer4_upsampled_bn = tf.layers.batch_normalization(layer4_upsampled, name="layer4_upsampled_bn")
    
    #1x1 Convolution on vgg_layer4_out   
    vgg_layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, filters=vgg_layer4_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer4_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer4_1x1_bn = tf.layers.batch_normalization(vgg_layer4_1x1, name="vgg_layer4_1x1_bn")
    
    #Final layer 4
    layer4_out = tf.add(vgg_layer4_1x1_bn, layer4_upsampled_bn, name='layer4_out')
    
    
    # Upsampled layer 3 using final layer 4
    layer3_upsampled4 = tf.layers.conv2d_transpose(layer4_out, vgg_layer3_out.get_shape().as_list()[-1], 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled')
    
    #batch normalization on upsampled layer
    layer3_upsampled4_bn = tf.layers.batch_normalization(layer3_upsampled4, name="layer3_upsampled4_bn")
    
        
    #layer3 taken from a 4x upsampling from layer7_1x1
    layer3_upsampled7 = tf.layers.conv2d_transpose(layer7_1x1, vgg_layer3_out.get_shape().as_list()[-1], 8, 
                                             strides= (4, 4), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3),
                                             name='layer3_upsampled7')
    
    layer3_upsampled7_bn = tf.layers.batch_normalization(layer3_upsampled7, name="layer3_upsampled7_bn")
    
    
    #1x1 Convolution on vgg_layer3_out
    vgg_layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, filters=vgg_layer3_out.get_shape().as_list()[-1], kernel_size=(1, 1), strides=(1, 1),
                                          name="vgg_layer3_1x1", activation=tf.nn.relu,
                                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                          kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #batch normalization on vgg layer
    vgg_layer3_1x1_bn = tf.layers.batch_normalization(vgg_layer3_1x1, name="vgg_layer3_1x1_bn")
    

    
    #Final Layer 3
    layer3_out = tf.add_n([vgg_layer3_1x1_bn, layer3_upsampled4_bn, layer3_upsampled7_bn], name='layer3_out')
    
    
    #Last Layer
    final_layer = tf.layers.conv2d_transpose(layer3_out, num_classes, 16,  
                                               strides= (8, 8), 
                                               padding= 'same', 
                                               kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
        
    return final_layer

tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=nn_last_layer, labels=correct_label),
                                        name="cross_entropy")
    regularization_loss = tf.losses.get_regularization_loss()
    total_loss = tf.add(cross_entropy_loss, regularization_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(total_loss)
    
    
    return logits , train_op, total_loss

tests.test_optimize(optimize)

def layers2(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):

    # Use a shorter variable name for simplicity
    layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

    # Apply 1x1 convolution in place of fully connected layer
    fcn8 = tf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")
    
    # Upsample fcn8 with size depth=(4096?) to match size of layer 4 so that we can add skip connection with 4th layer
    fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
    
    # Add a skip connection between current final layer fcn8 and 4th layer
    fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
    
    # Upsample again
    fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")
    
    # Add skip connection
    fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")
    
    # Upsample again
    fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

    return fcn11



def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())

    
    print("Training...")
    for i in tqdm(range(epochs)):
        print("EPOCH {} ...".format(i+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0005})
        print("Loss: = {:.3f}".format(loss))
    
    print("Training Finished!!")
    


def run():
    
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    model_dir = './models'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    
    ## gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        ## vgg_path = '.\\data\\vgg'
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        
        epochs = 20
        batch_size = 3

        # TF placeholders
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)

        nn_last_layer = layers2(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        ## sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)
        
        ## sess.run(tf.global_variables_initializer())
        ## saver = tf.train.Saver()

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(model_dir, runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image, saver)
        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()
print("Done")


TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
Tests Passed
Tests Passed
Tests Passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'./data\\vgg\\variables\\variables'


ResourceExhaustedError: OOM when allocating tensor of shape [7,7,512,4096] and type float
	 [[Node: fc6/weights/Adam/Initializer/zeros = Const[_class=["loc:@fc6/weights"], dtype=DT_FLOAT, value=Tensor<type: float shape: [7,7,512,4096] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'fc6/weights/Adam/Initializer/zeros', defined at:
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Si

In [3]:
import os.path

import tensorflow as tf

import helper

import warnings

from distutils.version import LooseVersion

import project_tests as tests





# Check TensorFlow Version

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)

print('TensorFlow Version: {}'.format(tf.__version__))



# Check for a GPU

if not tf.test.gpu_device_name():

    warnings.warn('No GPU found. Please use a GPU to train your neural network.')

else:

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))





def load_vgg(sess, vgg_path):

    """

    Load Pretrained VGG Model into TensorFlow.

    :param sess: TensorFlow Session

    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"

    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)

    """

    # TODO: Implement function

    #   Use tf.saved_model.loader.load to load the model and weights

    vgg_tag = 'vgg16'

    vgg_input_tensor_name = 'image_input:0'

    vgg_keep_prob_tensor_name = 'keep_prob:0'

    vgg_layer3_out_tensor_name = 'layer3_out:0'

    vgg_layer4_out_tensor_name = 'layer4_out:0'

    vgg_layer7_out_tensor_name = 'layer7_out:0'



    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)

    graph = tf.get_default_graph()

    input_layer = graph.get_tensor_by_name(vgg_input_tensor_name)

    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)

    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)

    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)

    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)



    return input_layer, keep_prob, layer3, layer4, layer7



tests.test_load_vgg(load_vgg, tf)





def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):

    """

    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.

    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output

    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output

    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output

    :param num_classes: Number of classes to classify

    :return: The Tensor for the last layer of output

    """

    weights_initializer_stddev = 0.01

    weights_regularized_l2 = 1e-3

    # TODO: Implement function

    # Convolutional 1x1 to mantain space information.

    conv_1x1_of_7 = tf.layers.conv2d(vgg_layer7_out,

                                     num_classes,

                                     1, # kernel_size

                                     padding = 'same',

                                     kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                     kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                     name='conv_1x1_of_7')

    # Upsample deconvolution x 2

    first_upsamplex2 = tf.layers.conv2d_transpose(conv_1x1_of_7,

                                                  num_classes,

                                                  4, # kernel_size

                                                  strides= (2, 2),

                                                  padding= 'same',

                                                  kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                                  kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                                  name='first_upsamplex2')

    conv_1x1_of_4 = tf.layers.conv2d(vgg_layer4_out,

                                     num_classes,

                                     1, # kernel_size

                                     padding = 'same',

                                     kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                     kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                     name='conv_1x1_of_4')

    # Adding skip layer.

    first_skip = tf.add(first_upsamplex2, conv_1x1_of_4, name='first_skip')

    # Upsample deconvolutions x 2.

    second_upsamplex2 = tf.layers.conv2d_transpose(first_skip,

                                                   num_classes,

                                                   4, # kernel_size

                                                   strides= (2, 2),

                                                   padding= 'same',

                                                   kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                                   name='second_upsamplex2')

    conv_1x1_of_3 = tf.layers.conv2d(vgg_layer3_out,

                                     num_classes,

                                     1, # kernel_size

                                     padding = 'same',

                                     kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                     kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                     name='conv_1x1_of_3')

    # Adding skip layer.

    second_skip = tf.add(second_upsamplex2, conv_1x1_of_3, name='second_skip')

    # Upsample deconvolution x 8.

    third_upsamplex8 = tf.layers.conv2d_transpose(second_skip, num_classes, 16,

                                                  strides= (8, 8),

                                                  padding= 'same',

                                                  kernel_initializer = tf.random_normal_initializer(stddev=weights_initializer_stddev),

                                                  kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2),

                                                  name='third_upsamplex8')

    return third_upsamplex8

tests.test_layers(layers)





def optimize(nn_last_layer, correct_label, learning_rate, num_classes):

    """

    Build the TensorFLow loss and optimizer operations.

    :param nn_last_layer: TF Tensor of the last layer in the neural network

    :param correct_label: TF Placeholder for the correct label image

    :param learning_rate: TF Placeholder for the learning rate

    :param num_classes: Number of classes to classify

    :return: Tuple of (logits, train_op, cross_entropy_loss)

    """

    # TODO: Implement function

    # create logits : 2D tensor where each row represents a pixel and each column a class.

    logits = tf.reshape(nn_last_layer, (-1, num_classes))

    correct_label = tf.reshape(correct_label, (-1,num_classes))

    # create loss function.

    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))

    # Define optimizer. Adam in this case to have variable learning rate.

    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)

    # Apply optimizer to the loss function.

    train_op = optimizer.minimize(cross_entropy_loss)



    return logits, train_op, cross_entropy_loss

tests.test_optimize(optimize)





def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,

             correct_label, keep_prob, learning_rate):

    """

    Train neural network and print out the loss during training.

    :param sess: TF Session

    :param epochs: Number of epochs

    :param batch_size: Batch size

    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)

    :param train_op: TF Operation to train the neural network

    :param cross_entropy_loss: TF Tensor for the amount of loss

    :param input_image: TF Placeholder for input images

    :param correct_label: TF Placeholder for label images

    :param keep_prob: TF Placeholder for dropout keep probability

    :param learning_rate: TF Placeholder for learning rate

    """

    # TODO: Implement function

    sess.run(tf.global_variables_initializer())



    print('Starting training... for {} epochs'.format(epochs))

    print()

    for epoch in range(epochs):

        print('Epoch : {}'.format(epoch + 1))

        loss_log = []

        for image, label in get_batches_fn(batch_size):

            _, loss = sess.run([train_op, cross_entropy_loss],

                                feed_dict={

                                    input_image: image,

                                    correct_label: label,

                                    keep_prob: 0.5,

                                    learning_rate: 0.00001

                                })

            loss_log.append('{:3f}'.format(loss))

        print(loss_log)

        print()

    print('Training finished')



tests.test_train_nn(train_nn)



def run():

    num_classes = 2

    image_shape = (160, 576)

    data_dir = './data'

    runs_dir = './runs'

    model_dir = './models'

    tests.test_for_kitti_dataset(data_dir)



    # Download pretrained vgg model

    helper.maybe_download_pretrained_vgg(data_dir)



    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.

    # You'll need a GPU with at least 10 teraFLOPS to train on.

    #  https://www.cityscapes-dataset.com/



    with tf.Session() as sess:

        # Path to vgg model

        vgg_path = os.path.join(data_dir, 'vgg')

        # Create function to get batches

        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)



        # OPTIONAL: Augment Images for better results

        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network



        # TODO: Build NN using load_vgg, layers, and optimize function



        # Placeholders

        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')

        learning_rate = tf.placeholder(tf.float32, name='learning_rate')



        # Getting layers from vgg.

        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)



        # Creating new layers.

        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)



        # Creating loss and optimizer operations.

        logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)



        # TODO: Train NN using the train_nn function

        epochs = 48 # 6 12 24 

        batch_size = 5



        saver = tf.train.Saver()



        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,

                 correct_label, keep_prob, learning_rate)



        # TODO: Save inference data using helper.save_inference_samples

        helper.save_inference_samples(model_dir, runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image, saver)



        # OPTIONAL: Apply the trained model to a video





if __name__ == '__main__':

    run()

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
Tests Passed
Tests Passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'./data\\vgg\\variables\\variables'


ResourceExhaustedError: OOM when allocating tensor with shape[7,7,512,4096]
	 [[Node: save/Assign_27 = Assign[T=DT_FLOAT, _class=["loc:@fc6/weights"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fc6/weights, save/RestoreV2_27/_7)]]

Caused by op 'save/Assign_27', defined at:
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Siddharth Singi\Miniconda3\envs\carnd-advdl-odlab\lib\site-